### LangChain Basics - Complex / Multiple Chains (LCEL Format)

In [1]:

import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_classic.chains import LLMChain, SequentialChain


In [2]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
model_name="gpt-3.5-turbo-0125"

In [3]:
llm = ChatOpenAI(
    model=model_name,
    temperature=0.0,
    max_tokens=4000,
    openai_api_key=openai_api_key
)

In [5]:
template1 = """
    You're a helpful assistant, who has good and proficient english writing skills.
    Write a Blog Outline for the following topic:
    Topic: {topic}
"""

template2 = """
    You're a helpful assistant, who has good and proficient english writing skills.
    Write a Blog Post based on the following outline:
    Outline: {outline}
"""

template3 = """
    You're a helpful assistant, who has good and proficient language skills including Translating.
    
    Translate the following text to Telugu:
    {article}
"""

prompt1 = PromptTemplate(
    input_variables=["topic"],
    template=template1
)

prompt2 = PromptTemplate(
    input_variables=["outline"],
    template=template2
)

prompt3 = PromptTemplate(
    input_variables=["article"],
    template=template3
)


In [ ]:
# Create LCEL chains using the pipe operator
chain1 = prompt1 | llm
chain2 = prompt2 | llm
chain3 = prompt3 | llm

# Combine chains using LCEL pipe operators
# Define reusable functions for data transformation
def extract_outline(x):
    return {"outline": x["outline"].content}

def prepare_article_input(x):
    return {
        "outline": x["outline"], 
        "article": x["article"].content
    }

def prepare_translation_input(x):
    return {
        "outline": x["outline"], 
        "article": x["article"], 
        "translated_text": x["translated_text"].content
    }

# Combine chains using LCEL pipe operators with reusable functions
sequential_chain = (
    {"topic": RunnablePassthrough()}
    | {"outline": chain1}
    | RunnableLambda(extract_outline)
    | {"outline": lambda x: x["outline"], "article": chain2}
    | RunnableLambda(prepare_article_input)
    | {"outline": lambda x: x["outline"], "article": lambda x: x["article"], "translated_text": chain3}
    | RunnableLambda(prepare_translation_input)
)


In [ ]:
# CONVENTIONAL WAY USING langchain-classic - Multiple Chains Together to form a Sequential Chain

# chain1 = LLMChain(llm=llm,
#                   prompt=prompt1, 
#                   output_key="outline")

# chain2 = LLMChain(llm=llm,
#                   prompt=prompt2,
#                   output_key="article")

# chain3 = LLMChain(llm=llm,
#                   prompt=prompt3,
#                   output_key="translated_text")

# overall_chain = SequentialChain(
#     chains=[chain1, chain2, chain3],
#     input_variables=["topic"],
#     output_variables=["outline", "article", "translated_text"],
#     verbose=True
# )

In [ ]:
# response = overall_chain({
#     "topic": "Corono Virus Management in India"
# })

# print(response)

In [7]:
response = sequential_chain.invoke("Microsoft Digital Transformation")

In [8]:
print("Outline:")
print(response["outline"])

Outline:
I. Introduction
    A. Definition of digital transformation
    B. Importance of digital transformation in today's business landscape

II. Microsoft's role in digital transformation
    A. Overview of Microsoft's history and evolution
    B. Microsoft's commitment to helping businesses with digital transformation

III. Microsoft's digital transformation solutions
    A. Azure cloud services
    B. Dynamics 365
    C. Power Platform

IV. Case studies of successful digital transformations with Microsoft
    A. Company A: Increased efficiency and productivity with Dynamics 365
    B. Company B: Seamless collaboration and communication with Azure cloud services
    C. Company C: Streamlined processes and improved decision-making with Power Platform

V. Benefits of partnering with Microsoft for digital transformation
    A. Access to cutting-edge technology
    B. Expert support and guidance
    C. Integration with existing Microsoft products and services

VI. Challenges and consid

In [9]:
print("\nBlog Post:")
print(response["article"])


Blog Post:
In today's fast-paced and ever-evolving business landscape, digital transformation has become a crucial aspect for companies looking to stay competitive and relevant. But what exactly is digital transformation? It refers to the integration of digital technology into all areas of a business, fundamentally changing how it operates and delivers value to customers. This shift is essential for businesses to adapt to changing market trends, customer preferences, and technological advancements.

Microsoft, a global leader in technology, has been at the forefront of digital transformation, helping businesses of all sizes navigate the complexities of this digital age. With a rich history and a commitment to innovation, Microsoft has developed a range of solutions to support businesses in their digital transformation journey.

One of Microsoft's key offerings is its Azure cloud services, providing businesses with a scalable and secure platform to host their applications and data. Thi

In [10]:
print("\nTranslated Text:")
print(response["translated_text"])


Translated Text:
ఈ రోజులో వేగవంతమైన మరియు ఎప్పుడూ వికసించే వ్యాపార పరిసరంలో, డిజిటల్ ట్రాన్స్ఫర్మేషన్ ప్రత్యేక ముఖ్యమైన అంశం అయింది కంపెనీలు పోటీదారులు ముఖ్యమైన అంశం అయిన కంపెటిటివ్ మరియు సంబంధితమైనవి ఉండటం కోసం. కానీ డిజిటల్ ట్రాన్స్ఫర్మేషన్ అంటే ఏమిటి అంటే? ఇది వ్యాపారంలో డిజిటల్ టెక్నాలజీని అన్ని ప్రకారాలుగా ఏర్పాటు చేసేందుకు సంబంధించినది, వ్యాపారం ఎలా పని చేస్తుందో మరియు విలువను గ్రాహకులకు ఇస్తుందో మూలంగా మారుతుంది. ఈ మార్పు వ్యాపారాలు మార్కెట్ ట్రెండ్లలో, గ్రాహక ఇష్టాలులో మరియు తాంత్రిక అధ్వాన్లతో అనుకూలంగా ఉండటం కోసం అత్యంత ముఖ్యమైనది.

టెక్నాలజీలో ప్రపంచ నాయకుడు మైక్రోసాఫ్ట్, డిజిటల్ ట్రాన్స్ఫర్మేషన్లో ముందుగా ఉండడంతో వ్యాపారాలను ఈ డిజిటల్ యుగంలో సహాయపడడంలో ముఖ్యమైన పాత్రను అందించింది. ఒక శ్రీమంత చరిత్ర మరియు ప్రజాస్వామ్యంకు ప్రతిజ్ఞ ఉండటంతో, మైక్రోసాఫ్ట్ వ్యాపారాలను తమ డిజిటల్ ట్రాన్స్ఫర్మేషన్ ప్రయాణంలో మద్దతు చేస్తుంది.

మైక్రోసాఫ్ట్ యొక్క ముఖ్య ఆఫర్లలో అదనపు ముఖ్యం అయినది అదనపు మైక్రోసాఫ్ట్ యొక్క అజూర్ క్లౌడ్ సేవలు, వ్యాపారాలకు అవిస్మరణీయమైన మరియు భద్రమైన ప్లాట్‌ఫారం అందిస్త